# Introduction to SciPy
Tutorial at EuroSciPy 2019, Bilbao
## 2. Signal analysis – Rocking motion of a TGV

In [1]:
import numpy as np
from scipy import fftpack, signal
%matplotlib notebook
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D

## Rocking motion of a TGV

Our data are stored in a compressed file `TGV_data.csv.bz2`. Each row of the uncompressed file contains entries separated by commas and the first row contains labels explaining the content of the respective column.

NumPy provides rather universal tools to import data from files into NumPy arrays. We will use `genfromtxt` which allows to deal with `bz2` compressed data files and also handles the labels in the first row of the data.

In [ ]:
np.info(np.genfromtxt)

In [ ]:
data = np.genfromtxt('data/TGV_data.csv.bz2', delimiter=',', names=True)

There are five columns identified by names:

In [ ]:
data.dtype.names

In [ ]:
time = data['Time_s']
omega_x = data['Gyroscope_x_rads']
omega_y = data['Gyroscope_y_rads']

Let us first get an idea of the data.

In [ ]:
plt.plot(time, omega_x)
plt.plot(time, omega_y)

### Spectral properties

Before analyzing the data any further, let us take a look at the time elapsed between subsequent measurements.

In [ ]:
time_intervals = time[1:]-time[:-1]

In [ ]:
_ = plt.hist(time_intervals, bins=100)

We will ignore these small differences and assume the data to be equally spaced in time with the mean time difference between subsequent data points.

In [ ]:
delta_t = np.mean(time_intervals)
delta_t

We will do the spectral analysis with the data for $\omega_y$.

In [ ]:
plt.plot(*signal.periodogram(omega_y, 1/delta_t))

In [ ]:
z = fftpack.rfft(omega_y)

In [ ]:
plt.plot(z[::2])

In [ ]:
plt.plot(z[1::2])

In [ ]:
freqs = fftpack.rfftfreq(omega_y.shape[0], delta_t)

In [ ]:
plt.plot(freqs[::2], z[::2])

In [ ]:
plt.plot(freqs[1::2], z[1::2])

We are interested in the signal around 1.4 Hz. Filter out frequencies beyond 2 Hz.

In [ ]:
filter_coeffs = signal.firwin(301, 3, pass_zero=True, fs=1/delta_t)

In [ ]:
freqs, response = signal.freqz(filter_coeffs, fs=1/delta_t)

In [ ]:
plt.plot(freqs, 20*np.log10(np.abs(response)))

In [ ]:
plt.polar(np.angle(response), np.abs(response))

In [ ]:
plt.plot(time, omega_y)

In [ ]:
omega_y_filtered = signal.convolve(omega_y, filter_coeffs, mode='valid')

In [ ]:
omega_y.shape, omega_y_filtered.shape

In [ ]:
plt.plot(time[150:-150], omega_y[150:-150])

In [ ]:
plt.plot(time[150:-150], omega_y_filtered)

Filter out anything but the range from 53-70 Hz

In [ ]:
filter_coeffs = signal.firwin(301, [53, 70], pass_zero='bandpass', fs=1/delta_t)

In [ ]:
freqs, response = signal.freqz(filter_coeffs, fs=1/delta_t)

In [ ]:
plt.plot(freqs, 20*np.log10(abs(response)))

In [ ]:
omega_y_filtered = signal.convolve(omega_y, filter_coeffs, mode='valid')

In [ ]:
plt.plot(time[150:-150], omega_y[150:-150])
plt.plot(time[150:-150], omega_y_filtered)

In [ ]:
plt.plot(fftpack.rfftfreq(omega_y_filtered.shape[0], delta_t)[::2],
         fftpack.rfft(omega_y_filtered)[::2])

In [ ]:
freq, sp_time, Sxx = signal.spectrogram(omega_y, fs=1/delta_t, nperseg=512)
plt.pcolormesh(freq, sp_time, Sxx)

In [ ]:
fig = plt.figure()
ax = Axes3D(fig)
ax.plot_surface(freq[:, np.newaxis], time, Sxx, cmap=cm.viridis)